<a href="https://colab.research.google.com/github/holictoweb/python_finance_analysis/blob/master/backtrader/toone_backtrading_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib as mat
import yfinance as yf
import backtrader as bt
from datetime import datetime, timedelta, timezone
 
from pyspark.sql import SparkSession
 
spark = SparkSession.builder.appName("toone_trader_01").getOrCreate()

1. ingest data from yfinance

In [ ]:
#7일 전까지만 지정 가능
targate_date = datetime.now(timezone.utc) + timedelta(hours=-9) - timedelta(days=7)
print(targate_date)
sdate = targate_date
sdate = datetime(2021, 4,20)
edate = datetime(2021, 4,22)
#edate = datetime.(2021, 3,31)

data = yf.download(  # or pdr.get_data_yahoo(...
  # tickers list or string as well
  tickers = "SPY AAPL MSFT",

  # use "period" instead of start/end
  # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
  # (optional, default is '1mo')
  period = "ytd",

  # fetch data by interval (including intraday if period < 60 days)
  # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
  # (optional, default is '1d')
  interval = "1m",

  # group by ticker (to access via data['SPY'])
  # (optional, default is 'column')
  group_by = 'ticker',

  # adjust all OHLC automatically
  # (optional, default is False)
  auto_adjust = True,

  # download pre/post regular market hours data
  # (optional, default is False)
  prepost = True,

  # use threads for mass downloading? (True/False/Integer)
  # (optional, default is True)
  threads = True,

  # proxy URL scheme use use when downloading?
  # (optional, default is None)
  proxy = None,
  start=sdate 
  #end=edate
  )

2021-04-18 07:38:54.913993+00:00
[*********************100%***********************]  3 of 3 completed


In [ ]:
data

MSFT                  ...    AAPL               
                             Open    High     Low  ...     Low   Close Volume
Datetime                                           ...                       
2021-04-20 04:00:00-04:00  259.00  259.00  258.99  ...  134.98  135.02    0.0
2021-04-20 04:01:00-04:00  259.34  259.34  259.34  ...  135.02  135.03    0.0
2021-04-20 04:02:00-04:00  258.90  259.23  258.90  ...  135.00  135.03    0.0
2021-04-20 04:03:00-04:00  258.99  259.23  258.83  ...  134.38  135.02    0.0
2021-04-20 04:04:00-04:00  259.24  259.24  258.92  ...  134.44  134.69    0.0
...                           ...     ...     ...  ...     ...     ...    ...
2021-04-23 19:55:00-04:00  261.75  261.77  261.75  ...  134.26  134.26    0.0
2021-04-23 19:56:00-04:00  261.77  261.77  261.68  ...  134.26  134.28    0.0
2021-04-23 19:57:00-04:00  261.77  261.77  261.61  ...  134.26  134.28    0.0
2021-04-23 19:58:00-04:00  261.65  261.98  261.59  ...  134.27  134.28    0.0
2021-04-23 19:59:00-04:00  261.65  261.96  261.56  ...  134.27  134.28    0.0

[3837 rows x 15 columns]

In [ ]:
data['MSFT']

,Open,High,Low,Close,Volume
Datetime,,,,,
2021-04-20 04:00:00-04:00,259.00,259.00,258.99,258.99,0.0
2021-04-20 04:01:00-04:00,259.34,259.34,259.34,259.34,0.0
2021-04-20 04:02:00-04:00,258.90,259.23,258.90,259.23,0.0
2021-04-20 04:03:00-04:00,258.99,259.23,258.83,258.99,0.0
2021-04-20 04:04:00-04:00,259.24,259.24,258.92,258.99,0.0
...,...,...,...,...,...
2021-04-23 19:55:00-04:00,261.75,261.77,261.75,261.77,0.0
2021-04-23 19:56:00-04:00,261.77,261.77,261.68,261.75,0.0
2021-04-23 19:57:00-04:00,261.77,261.77,261.61,261.77,0.0


2. save to spark lakehouse

In [ ]:
ticker_list = ["005930.KS", '068270.KS', '035420.KS', '064260.KQ']
 
for ticker in ticker_list:
  # pandas df 로 받은 데이터를 spark dataframe 으로 변경
  pdf = yf.download(ticker, sdate='2020-01-01')
  df = spark.createDataFrame(pdf)  #df.show()
  
  #df.write.format('parquet').save('drive/MyDrive/data-warehouse/test')  # column Adj Close 와 관련한 이슈 발생
  # AnalysisException: Attribute name "Adj Close" contains invalid character(s) among " ,;{}()\n\t=". Please use alias to rename it. 
 
  ticker = ticker.split('.')[0]
  df = df.withColumnRenamed("Adj Close", "AdjClose").withColumn("Code", lit(ticker))
  
  # 1. unmanaged table 구성 
  #    parquet 으로 먼저 다운 받은 데이터를 가지고 테이블을 구성. 테이블 삭제 시 해당 데이터 파일은 삭제 되지 않음
  table_path = '/content/drive/MyDrive/data-warehouse/stock_day/'
 
  # Overwrite specific partitions in spark dataframe write method
  # df.write.mode(SaveMode.Overwrite).save("/root/path/to/data/partition_col=value")
  df.write.format('parquet').mode("overwrite").save(table_path + 'Code='+ ticker)
  df.show()
  create_table_sql = 'create table if not exists stocklab.stock_day using org.apache.spark.sql.parquet options (path "'+ table_path +'")'
  #print(create_table_sql)
  spark.sql(create_table_sql)
 
  # 2. managed table 구성
  # 현재 DB 구성 하위로 테이블을 생성 하며 df에서 db 상에 테이블을 생성하는 방식으로 테이블과 데이터가 함께 관리 되며 삭제도 함께 진행. 
  #df.write.mode("overwrite").partitionBy("Code").saveAsTable("mng_stock_day")

In [ ]:
# 기본 환경 구성 
 
!pip install pyspark
!pip install yfinance
!pip install backtrader
 
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/MyDrive/data-warehouse/

     |████████████████████████████████| 212.3MB 71kB/s 
     |████████████████████████████████| 204kB 45.9MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=cb45a425d6ecaabc40062a58e313f1a679c09bc495b1b9b36b951dd4818422db
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
     |████████████████████████████████| 5.5MB 7.3MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23442 sha256=5b1bc2f29fada4f9891762e11027760dea257c65fe436f96ce9837503ea009ca
  Stored in directory: /root/.cache/pip/wheels/f8/2a/0f/4b5a86e1d52e451757eb6bc17fd899629f0925c777741b6d04
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
     |████████████████████████████████| 419kB 4.9MB/s 
